In [1]:
import pandas as pd
import sqlite3
from tabulate import tabulate
import sqlite3
import os
import csv
import re
import shutil

In [3]:
keywords = [
    "racism",
    "racial justice",
    "police brutality",
    "police violence",
    "black people",
    "black person",
    "george floyd",
    "breonna taylor",
    "trayvon martin",
    "ahmaud arbery",
    "sandra bland",
    "tamir rice",
    "eric garner",
    "black lives matter",
    "systemic racism",
    "structural racism",
    "racial inequality",
    "racial profiling",
    "racial bias",
    "institutional racism",
    "mass incarceration",
    "criminal justice reform", 
    "abolish the police",
    "defund the police", 
    "racial uprising",
    "white supremacy"
]

In [ ]:
like_conditions = " OR ".join([f"LOWER(transcript) LIKE '%{kw.lower()}%'" for kw in keywords])

query = f"""
SELECT epID, episodeDateLocalized FROM podcast_episodes
WHERE {like_conditions}
"""

db_path = "../data/data.db"
output_csv = "../data/race_epIDs.csv"

with sqlite3.connect(db_path) as connection:
    df = pd.read_sql_query(query, connection)

df.to_csv(output_csv, index=False)

Get a list of all episodes and save relevant episode level data

In [ ]:
episodes_dir = '../data/episodes'
ep_ids = [
    (int(os.path.splitext(filename)[0]),)
    for filename in os.listdir(episodes_dir)
    if filename.endswith('.csv') and filename[:-4].isdigit()
]
len(ep_ids)

152539

In [ ]:
db_path = '../data/data.db'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute('DROP TABLE IF EXISTS temp_epids')
cursor.execute('CREATE TEMP TABLE temp_epids (epID INTEGER PRIMARY KEY)')
cursor.executemany('INSERT INTO temp_epids (epID) VALUES (?)', ep_ids)
conn.commit()

query = '''
SELECT podcast_episodes.*
FROM podcast_episodes
JOIN temp_epids ON podcast_episodes.epID = temp_epids.epID
'''
df = pd.read_sql_query(query, conn)

df.to_csv('../data/episodeLevelData.csv', index=False)
conn.close()

print(f"Saved {len(df)} rows to ../data/episodeLevelData.csv")

Saved 152539 rows to ../data/episodeLevelData.csv


Move all episodes that do not discuss race to a different folder

In [7]:
valid_ids = set(df['epID'].astype(str))  

episodes_dir = '../data/episodes/'
bad_episodes_dir = '../data/bad_episodes/'

for filename in os.listdir(episodes_dir):
    if filename.endswith('.csv'):
        file_id = filename.replace('.csv', '')
        if file_id not in valid_ids:
            # Move the file to the bad_episodes directory
            src_path = os.path.join(episodes_dir, filename)
            dst_path = os.path.join(bad_episodes_dir, filename)
            shutil.move(src_path, dst_path)

Move all episodes that dicuss race but do not contain any collective action sentences to the bad folder

In [8]:
for filename in os.listdir(episodes_dir):
    if filename.endswith('.csv'):
        file_path = os.path.join(episodes_dir, filename)
        try:
            chunks = pd.read_csv(file_path, usecols=['collectiveAction'], chunksize=10000)
            has_zero = any((chunk['collectiveAction'] == 0).any() for chunk in chunks)
        except Exception as e:
            print(f"Error reading {filename}: {e}")
            has_zero = False

        if not has_zero:
            shutil.move(file_path, os.path.join(bad_episodes_dir, filename))